In [14]:
# Check how many processes are available on your machine
import multiprocessing
multiprocessing.cpu_count()

20

In [2]:
import praw
import json
import string


# function to preprocess text for Misra-Gries algorithm
import re
def preprocess(text):
    # define regular expression pattern to match links, numbers, and punctuation
    pattern = re.compile(r'\bhttps?://\S+|\b\d+\b|\b\w+')
    # split the text into a list of words and apply the transformations
    words = [word.lower().replace('...', '').replace('\n', '').replace('\t', '').replace('’', '').encode('ascii', 'ignore').decode('ascii') for word in pattern.findall(text) if not word.isdigit()]
    # join the words back into a single string with spaces
    return ' '.join(words)



def process_subreddit(subreddit):
    subreddit_top_comments = []
    for post in subreddit.hot(limit=100):
        comments = post.comments.list()[:100]
        for comment in comments:
            if isinstance(comment, praw.models.MoreComments):
                continue
            subreddit_top_comments.append(comment.body)
    # preprocess text
    subreddit_top_comments = [preprocess(comment) for comment in subreddit_top_comments]
    # write to file with subreddit name
    with open(subreddit.display_name + '.txt', 'w') as f:
        for comment in subreddit_top_comments:
            f.write(comment + '\n')


# load credentials from file
with open('client_secret.json') as f:
    data = json.load(f)
    client_id = data['client_id']
    client_secret = data['client_secret']
    user_agent = data['user_agent']

reddit = praw.Reddit(client_id=client_id, client_secret=client_secret, user_agent=user_agent)

# get the top 25 subreddits minus the first 1
subreddits = list(reddit.subreddits.popular(limit=25))[1:]


for subreddit in subreddits:
    process_subreddit(subreddit)




